In [1]:
import thermox
import jax
import jax.numpy as jnp
from jax.scipy.linalg import solve, inv, expm

### In this notebook we show how to run three basic thermodynamic algorithms:
1. Thermodynamic linear solver: find $x$ such that $Ax = b$,
2. Thermodynamic matrix inverse: find $A^{-1}$,
3. Thermodynamic matrix exponential: find $\exp{(A)}$.

These algorithms are all based on extracting statistical information from the multivariate Ornstein-Uhlenbeck process, defined as
$$ dx = - A(x - b) dt + \mathcal{N}(0, 2D) $$

Let us start with solving a linear system $Ax = b$. In this case, $D = \mathbb{I}$.

In [2]:
key = jax.random.PRNGKey(42) # random PRNG key
dimension = 50 # problem size
mean = jnp.zeros(dimension) # mean vector
n = 2 * dimension # number of degrees of freedom
A = jax.random.normal(key, shape=(dimension, 2*dimension,))
A = (A @ A.T) / n # random positive-semi definite matrix from the Wishart distribution
b = jax.random.normal(key, shape=(dimension,))

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
x_s = thermox.linalg.solve(A, b)

We know look at the absolute error $||x_s - x^*||$, using `scipy`'s `solve` function to get the exact solution $x^*$:

In [4]:
print(r"||x_s - x*|| = ", jnp.linalg.norm(x_s - solve(A,b)))

||x_s - x*|| =  0.2386516


## Thermodynamic matrix inverse

This time, no need to define the vector $b$. The matrix is simply defined as the continuous-time correlation matrix
 $$A^{-1} \approx C(t,t') = \langle x(t) x(t')\rangle$$

In [5]:
thermo_inv = thermox.linalg.inv(A)

In [6]:
print("||A^{-1} - C(t,t')|| =", jnp.linalg.norm(inv(A) - thermo_inv))

||A^{-1} - C(t,t')|| = 1.5911188


Let's increase the number of samples to get a better inverse and look at the error again.

In [7]:
thermo_inv = thermox.linalg.inv(A, num_samples=100000)
print("||A^{-1} - C(t,t)|| =", jnp.linalg.norm(inv(A) - thermo_inv))

||A^{-1} - C(t,t)|| = 0.46243933


It went down! And gathering 10 times more samples only took about twice the time.

Let us now consider matrix exponentials. Due to the way we obtain the matrix exponentials, the negative exponential $\exp{(-A)}$ is more easily gathered. This is because the autocovariance function is equal to $\exp{(-A)}$, when we have $A$ as the drift term of the SDE.

$$ C(t+\tau, t) = \frac{1}{T} \int_{t_0}^{t_0+T} dt x(t+\tau) x^\intercal(t) = \exp{(-A \tau)} $$

In [8]:
thermo_negexp = thermox.linalg.negexpm(A)


In [9]:
print(r"||exp(-A) - C(t+tau,t)||=", jnp.linalg.norm(expm(-A) - thermo_negexp))

||exp(-A) - C(t+tau,t)||= 11.980379


We once again increase the number of samples, which brings the error down.

In [10]:
thermo_negexp = thermox.linalg.negexpm(A, num_samples=100000, dt=1)
print(r"||exp(-A) - C(t+tau,t)||=", jnp.linalg.norm(expm(-A)- thermo_negexp))

||exp(-A) - C(t+tau,t)||= 11.706923


## TODO?? : add notes and code about non-positive matrices for matrix exponentials